In [44]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

import os

import openpyxl
from os import listdir
from os.path import isfile, join

from sklearn.metrics import accuracy_score

dataDir = './archive/images'
anotation_path = './archive/annotations'

In [56]:
# CLAHE

def clahe(testImg):
    testImg = cv2.cvtColor(testImg, cv2.COLOR_RGB2HSV)

    h, s, v = testImg[:,:,0], testImg[:,:,1], testImg[:,:,2]

    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))

    v = clahe.apply(v)

    testImg = np.dstack((h, s, v))

    testImg = cv2.cvtColor(testImg, cv2.COLOR_HSV2RGB)
    
    return testImg

# REMOVE SHADOWS

def removeShadows(testImg):
    
    rgb_planes = cv2.split(testImg)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)

    testImg = cv2.merge(result_norm_planes)
    
    return testImg

#

def redMask(testImg):
    testImg = cv2.cvtColor(testImg, cv2.COLOR_RGB2HSV)

    # lower mask (0-10)
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(testImg, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(testImg, lower_red, upper_red)

    # join my masks
    mask = mask0+mask1

    # set my output img to zero everywhere except my mask
    testImgRed = ogImg.copy()
    
    testImgRed = cv2.cvtColor(testImgRed, cv2.COLOR_RGB2HSV)
    
    testImgRed[np.where(mask==0)] = 0
    
    testImgRed = cv2.cvtColor(testImgRed, cv2.COLOR_HSV2RGB)
    
    testImg = cv2.cvtColor(testImg, cv2.COLOR_HSV2RGB)
    
    return testImgRed


# CLOSING

def closing(testImgRed):
    testImgRed = cv2.GaussianBlur(testImgRed, (11,11), 3)
    testImgGray = cv2.cvtColor(testImgRed, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    testImgGray = cv2.morphologyEx(testImgGray, cv2.MORPH_CLOSE, kernel)

    return testImgGray



def findCircles(testImgRed):
    # Use the Hough transform to detect circles in the image
    circles = cv2.HoughCircles(testImgRed, cv2.HOUGH_GRADIENT, 1, testImgRed.shape[0] / 8, param1=50, param2=45, minRadius=0, maxRadius=0)
    # If we have extracted a circle, draw an outline
    # We only need to detect one circle here, since there will only be one reference object
    return circles

def detectCircles(ogImage):
    finalImg = ogImg.copy()
    testImg = ogImg.copy()
            
    for i in range(3):
        testImg = ogImg.copy()
        if i == 0:
            noShadows = removeShadows(testImg)
            redOnly = redMask(noShadows)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)        
        if i == 1:
            claheImg = clahe(testImg)
            redOnly = redMask(claheImg)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)
        if i == 2:
            redOnly = redMask(testImg)
            grayClosing = closing(redOnly)
            circles = findCircles(grayClosing)
        if circles is not None:
            return circles.shape[1]
    return 0
    
    


In [ ]:
onlyfiles = [f for f in listdir(dataDir) if isfile(join(dataDir, f))]
 
workbook = openpyxl.load_workbook("RoadSigns.xlsx")
 
sheet = workbook.active
# Start from the first cell.
# Rows and columns are zero indexed.
row_number = 2

 
# iterating through content list
for item in onlyfiles :
    
    ogImg = cv2.imread(os.path.join(dataDir, item))

    redSigns = 0;
    
    ogImg = cv2.cvtColor(ogImg, cv2.COLOR_BGR2RGB)
    
    nrCircles = detectCircles(ogImg)
    
    sheet.cell(row = row_number, column = 6, value = nrCircles)

    row_number += 1

workbook.save("RoadSigns.xlsx")

In [58]:
actual = sheet['C']
detected = sheet['F']

def mapping(cell):
    return cell.value

actual = list(map(mapping, actual[1:]))
detected = list(map(mapping, detected[1:]))

accuracy = round(accuracy_score(actual, detected) * 100)

print("Accuracy for Red Circles: {}%".format(accuracy))


Accuracy for Red Circles: 56%


In [22]:
image_path = "./archive/images"
anotation_path = "./archive/annotations"
# import xlsxwriter module
import xlsxwriter
import xml.dom.minidom
import os

from os import listdir
from os.path import isfile, join
onlyfiles = [".".join(f.split(".")[:-1]) for f in listdir(image_path) if isfile(join(image_path, f))]
 
workbook = xlsxwriter.Workbook('RoadSigns.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'File Name')
worksheet.write('B1', 'Stop Sign Count')
worksheet.write('C1', 'Round Red Sign Count')
worksheet.write('D1', 'Blue Square Sign Count')
# Start from the first cell.
# Rows and columns are zero indexed.
row = 1
column = 0

 
# iterating through content list
for item in onlyfiles :
    stops = 0;
    blueSigns = 0;
    redSigns = 0;
    doc = xml.dom.minidom.parse(os.path.join(anotation_path,item + ".xml"));
    
    signsNames = doc.getElementsByTagName("name")
    
    for sign in signsNames:
     if sign.childNodes[0].nodeValue == "stop":
        stops += 1
     elif sign.childNodes[0].nodeValue == "speedlimit":
        redSigns += 1
     elif sign.childNodes[0].nodeValue == "crosswalk":
        blueSigns += 1
    
    # write operation perform
    worksheet.write(row, column, item)
    worksheet.write(row, 1, stops)
    worksheet.write(row, 2, redSigns)
    worksheet.write(row, 3, blueSigns)
 
    # incrementing the value of row by one
    # with each iterations.
    row += 1
     
workbook.close()